# Get book characters 

In [3]:
import pandas as pd
import string
from bs4 import BeautifulSoup
from urllib.request import urlopen

#web scraping list of characters from witcher wiki
#https://witcher.fandom.com/wiki/Category:Characters_in_the_stories
from bs4 import BeautifulSoup
from urllib.request import urlopen

book_characters = {}

def get_characters(book, url):
    book_characters[book] = set()
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")

    images = soup.find_all("img")

    for image in images:
        name = image["alt"]
        name = name.split(" ")[0]
        if name not in book_characters[book]:
            if not name == "Witcher" and not name == "Cirilla":
                book_characters[book].add(name)

# I
get_characters('I' ,"https://witcher.fandom.com/wiki/Category:Blood_of_Elves_characters")
# II
get_characters('II', "https://witcher.fandom.com/wiki/Category:Time_of_Contempt_characters")
# III
get_characters('III', "https://witcher.fandom.com/wiki/Category:Baptism_of_Fire_characters")
# IV
get_characters('IV', "https://witcher.fandom.com/wiki/Category:The_Tower_of_the_Swallow_characters")
# V
get_characters('V', "https://witcher.fandom.com/wiki/Category:The_Lady_of_the_Lake_characters")

In [4]:
for book, list_characters in book_characters.items():
    print(book)
    print(sorted(list_characters))
    print(len(list_characters))
    print('\n')

I
['Aelirenn', 'Anséis', 'Artaud', 'Bronibor', 'Cahir', 'Calanthe', 'Carduin', 'Ciri', 'Coën', 'Crach', 'Dandelion', 'Demavend', 'Donimir', 'Duke', 'Eist', 'Ekkehard', 'Emhyr', 'Eskel', 'Essi', 'Esterad', 'Ethain', 'Everett', 'Fercart', 'Foltest', 'Francesca', 'Geralt', 'Gerhart', 'Henselt', 'Herbert', 'Hereward', 'Iola', 'Ithlinne', 'Jarre', 'Keira', 'Lambert', 'Lydia', 'Lytta', 'Maria', 'Menno', 'Meve', 'Nenneke', 'Niedamir', 'Ori', 'Paulie', 'Pavetta', 'Philippa', 'Podveyka', 'Rayla', 'Regan', 'Rience', 'Roach', 'Shani', 'Sheldon', 'Sigismund', 'Tissaia', 'Triss', 'Vanielle', 'Venzlav', 'Vesemir', 'Vilgefortz', 'Villem', 'Virfuril', 'Vissegerd', 'Windhalm', 'Xavier', 'Yannick', 'Yarpen', 'Yennefer', 'Zyvik']
69


II
['Akerspaark', "Akerspaark's", 'Aplegatt', 'Ardal', 'Artaud', 'Asse', 'Blue', 'Bronibor', 'Cahir', 'Calanthe', 'Carduin', 'Ceallach', 'Ciri', 'Codringher', 'Coën', 'Dandelion', 'Demavend', 'Detmold', 'Dorregaray', 'Eist', 'Eithné', 'Elan', 'Emhyr', 'Eredin', 'Eskel', 'Es

# Get chapters from file txt

In [13]:
import os
books = ['I - Blood of Elves', 'II - Times of Contempt', 'III - Baptism of Fire', 'IV - The Tower of the Swallow', 'V - The Lady of the Lake']
book_chapters = {}

for book in books:
    book_chapters[book] = set()
    chapters = os.listdir('./book_chapters/' + book + '/txt/')
    for chapter in chapters:
        book_chapters[book].add(int(chapter.replace("chapter-", "").replace(".txt", "")))

In [11]:
for book, chapters in book_chapters.items():
    print(book)
    print(chapters)

I - Blood of Elves
{1, 2, 3, 4, 5, 6, 7}
II - Times of Contempt
{1, 2, 3, 4, 5, 6, 7}
III - Baptism of Fire
{1, 2, 3, 4, 5, 6, 7}
IV - The Tower of the Swallow
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
V - The Lady of the Lake
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}


# Get characters for chapters

In [23]:
def get_characters_for_chapters(chapter, name_book):
    chapter_characters = set()
    lines = []
    for line in chapter:
        lines.append(line)

    table = str.maketrans('', '', string.punctuation)
    stripped = [l.translate(table) for l in lines]
    
    for line in stripped:
        for charater in book_characters[name_book]:
            if charater in line:
                chapter_characters.add(charater)
                #print(chapter_characters)
        
    return chapter_characters

In [27]:
def get_column(chapter_characters, name_book):
    l = []
    for character in book_characters[name_book]:
        if character in chapter_characters:
            l.append(1)
        else:
            l.append(0)
    return l

# Create final csv 

In [70]:
import pandas as pd

for book, list_chapters in book_chapters.items():
    data = {}
    name_book = book.split(" ", 1)[0]
    for chapter in list_chapters:
        path_chapter = "./book_chapters/" + book + "/txt/chapter-" + str(chapter) + ".txt"
        data[chapter] = get_column(get_characters_for_chapters(open(path_chapter, "r"), name_book), name_book)
    
    df = pd.DataFrame(data, index=book_characters[name_book])
    
    # remove characters that not appear in 50% of chapters
    for row in df.iterrows():
        if row[1].sum() < len(list_chapters)/2:
            df = df.drop(row[0])
    
    # change name of columns
    columns = []
    for column in df.columns:
        columns.append('c' + str(column))
    df.columns = columns
    
    # sort df
    df = df.sort_values(by = columns, ascending = False)

    df.to_csv("./book_chapters/" + book + "/final-book-" + name_book + ".csv")    

# Create json

In [14]:
import pandas as pd

for book in books:
    current_book = pd.read_csv(".books//book_chapters/" + book + "/final-book-" + book.split(" ", 1)[0] + ".csv")
    dict = {}
    i = 1
    l = []
    for column in current_book.columns[1:]:
        l1 = {}
        l1['chapter'] = int(column.replace('c', ''))
        dict[column] = []
        for row in current_book.iterrows():
            # name = row[1][0]
            l1[row[1][0]] = row[1][i]
            if row[1][i] == 1:
                dict[column].append(row[1][0])
                # l1.append(row[1][0])
        i += 1
        l.append(l1)
    with open('./app/assets/book-' + book.split(" ", 1)[0] + '.json', 'w') as f:
        json.dump(l, f)


In [ ]:
import json
app_json = json.dumps(l)
print(app_json)